In [1]:
import pandas as pd
import re
import json
import requests
import numpy as np
from IPython.display import HTML, display_html, display
from bs4 import BeautifulSoup as bs

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
#DataSet sucio, descargado de Kaggle 'https://www.kaggle.com/irinachuchueva/russian-wholesale-electricity-market'
rawdataset=pd.read_csv('RU_Electricity_Market_UES_dayahead_price.csv',encoding="latin1")
rawdataset.head()

,timestep,UES_Northwest,UES_Siberia,UES_Middle_Volga,UES_Urals,UES_Center,UES_South
0,2017-01-01 00:00,964.67,1006.27,1032.80,953.23,1050.17,1706.30
1,2017-01-01 01:00,984.79,1003.79,1033.34,946.79,1051.15,1592.17
2,2017-01-01 02:00,962.54,1005.09,1029.51,944.35,1039.84,1561.85
3,2017-01-01 03:00,888.44,1002.38,976.34,906.15,977.04,1534.37
4,2017-01-01 04:00,847.10,1002.58,945.28,894.08,938.27,1517.58


In [3]:
#DataFrame limpio, separo la hora de los dias y agrego una columna que indica el año del dato
Russiaraw=rawdataset.copy()
Russiaraw['DateTime']=Russiaraw.timestep.str.split(" ")
Russia=Russiaraw.assign(Date=Russiaraw.DateTime.str[0],Time=Russiaraw.DateTime.str[1])
Russia=Russia[['Date','Time','UES_Northwest','UES_Siberia','UES_Middle_Volga','UES_Urals','UES_Center','UES_South']]
Russia['Year']=Russia.Date.str.split("-")
Russia['Year']=Russia.Year.str[0]
#Obtengo las regiones de Rusia
Ruregions=[]
for e in Russia.columns:
    if e.startswith('UES'):
        Ruregions.append(e)

In [4]:
#Datos de la API, 'https://www.eia.gov/opendata/qb.php?category=2134404&sdid=INTL.2-2-MEX-BKWH.A'
with open('API_EIA_key.txt', 'r') as f:
    acceso=f.readlines()[0].split('\n')
key=acceso[0]
#Agrego una key que me proporciono el sitio y que esta un archivo indicado en el .gitignore
url='http://api.eia.gov/series/?api_key='+key+'&series_id=INTL.2-2-MEX-BKWH.A'
res=requests.get(url)
res

<Response [200]>

In [5]:
#Del API solo obtengo los datos por año de el consumo promedio de electricidad en México
results=res.json()
rawAPI1=results['series'][0]['data']

In [6]:
#Tomare datos de la lista del elemento 1 al 7
Mexico_per_year=[]
for e in range(1,8):
    Mexico_per_year.append(rawAPI1[e][1])
#Los enlisto en forma inversa a como vienen del dataset    
Mexico_per_year.reverse()

In [7]:
#Calcule con ecuacion de la recta con datos de la API de México
def mx_mean_calc(x):
    m=(float(rawAPI1[1][1])-float(rawAPI1[2][1]))/(int(rawAPI1[1][0])-int(rawAPI1[2][0]))
    b=(float(rawAPI1[1][1]))-m*(int(rawAPI1[1][0]))
    return m*x+b

In [8]:
#Calcule los tres ultimos años que faltan al dataset para México
Mexico_per_year.append(round(mx_mean_calc(2017),5))
Mexico_per_year.append(round(mx_mean_calc(2018),5))
Mexico_per_year.append(round(mx_mean_calc(2019),5))
#Lista de los años que deseo en el DF
Mexico_years=[]
for e in range(2010,2020):
    Mexico_years.append(e)

In [9]:
#DataFrame de México del año 2010 al 2019
Mexico_per_year_mean=pd.DataFrame(list(zip(Mexico_years,Mexico_per_year)),columns =['Year','Mexico']) 

In [10]:
#Calcule con ecuacion de la recta con datos del Dataset del 2017 y 2018 para Rusia
def ru_mean_calc(x):
    m=(ru_mean(2018)-ru_mean(2017))/(2018-2017)
    b=ru_mean(2018)-m*2018
    return m*x+b

In [11]:
#Solo existen datos de Rusia del 2017 al 2019, funcion que calcula el consumo promedio de toda Rusia, determinado año
def ru_mean(year):
    m1=Russia[Russia['Year']==str(year)].UES_Northwest.mean()
    m2=Russia[Russia['Year']==str(year)].UES_Siberia.mean()
    m3=Russia[Russia['Year']==str(year)].UES_Middle_Volga.mean()
    m4=Russia[Russia['Year']==str(year)].UES_Urals.mean()
    m5=Russia[Russia['Year']==str(year)].UES_Center.mean()
    m6=Russia[Russia['Year']==str(year)].UES_South.mean()
    return (m1+m2+m3+m4+m5+m6)/len(Ruregions)

In [12]:
#Calcule 2010 a 2016 con ecuacion de la recta, puesto que no viene en el dataset, y agrego a una lista
Russia_per_year=[]
for e in range(2010,2017):
    Russia_per_year.append(ru_mean_calc(e))
#Calcule 2017 y 2018 de acuerdo al data set que descargue y agrego a una lista
Russia_per_year.append(ru_mean(2017))
Russia_per_year.append(ru_mean(2018))
#Calculamos 2019 con ecuacion de la recta y agrego a una lista
Russia_per_year.append(ru_mean_calc(2019))
#Lista de los años que deseo en el DF
Russia_years=[]
for e in range(2010,2020):
    Russia_years.append(e)
#DataFrame de Rusia del año 2010 al 2019
Ru={'Year':Russia_years,'Russia':Russia_per_year}
Russia_per_year_mean=pd.DataFrame(Ru)

In [13]:
#Mezclo los dos dataframes, el de Mexico (de la API) y de Rusia (del DataSet)
dftwo=pd.merge(Mexico_per_year_mean,Russia_per_year_mean, on=['Year','Year'])

In [14]:
#Datos de la API, 'https://www.eia.gov/opendata/qb.php?category=2134404&sdid=INTL.2-2-JPN-BKWH.A'
with open('API_EIA_key.txt', 'r') as f:
    acceso=f.readlines()[0].split('\n')
key=acceso[0]
#Agrego una key que me proporciono el sitio y que esta un archivo indicado en el .gitignore
url='http://api.eia.gov/series/?api_key='+key+'&series_id=INTL.2-2-JPN-BKWH.A'
res=requests.get(url)
res

<Response [200]>

In [15]:
#Del API solo obtengo los datos por año de el consumo promedio de electricidad en Japon
#El 27 de febrero de 2010 en Japón, un terremoto de magnitud 7,3 en la escala de Richter a las 5:30 (hora local) esto 
#afecto el consumo de electricidad a nivel país
results=res.json()
rawAPI2=results['series'][0]['data']
#rawAPI2 tiene 37 elementos, de los cuales calcularemos la ecuacion de la recta por regresion lineal

In [16]:
#Hago dos listas de la electriciad consumida en Japon (jpelec) y de los años (jpyear)
jpelec=[]
for e in range(1,5):
    jpelec.append(rawAPI2[e][1])
jpyear=[]
for e in range(1,5):
    jpyear.append(rawAPI2[e][0])
jpelec.reverse()    
jpyear.reverse()

In [17]:
#Calcule con regresion lineal la ecuacion de la recta con datos de la API de Japón
def jp_mean_calc(x,y): 
    n=np.size(x) 
    mx,my=np.mean(x),np.mean(y) 
    SSxy=np.sum(y*x)-n*my*mx 
    SSxx=np.sum(x*x)-n*mx*mx 
    m=SSxy/SSxx 
    b=my-m*mx 
    return(m,b) 
#De la API hice dos listas, años y consumo, esas dos las convierto en un arreglo de numpy
x=np.array([int(e) for e in jpyear])
y=np.array([float(f) for f in jpelec])
#b es un tupla con la pendiente y la ordenada al origen de la recta calculadas de la regresion lineal
b=jp_mean_calc(x,y)

In [18]:
#Calcule los tres ultimos años que faltan al dataset para Japón con la pendiente y la ordenada al origen
Japan_per_year=[]
for e in range(1,8):
    Japan_per_year.append(rawAPI2[e][1])
Japan_per_year.reverse()    
Japan_per_year.append(round(b[0]*2017+b[1],5))
Japan_per_year.append(round(b[0]*2018+b[1],5))
Japan_per_year.append(round(b[0]*2019+b[1],5))
#Lista de los años que deseo en el DF
Japan_years=[]
#Tenemos 37 elementos en la lista de años
for e in range(1,8):
    Japan_years.append(rawAPI2[e][0])
Japan_years.reverse()    
Japan_years.append('2017')    
Japan_years.append('2018')    
Japan_years.append('2019')  
Japan_years=[int(e) for e in Japan_years]

In [19]:
#DataFrame de Japón del año 2010 al 2019
Japan_per_year_mean=pd.DataFrame(list(zip(Japan_years,Japan_per_year)),columns =['Year','Japan']) 

In [20]:
#Mezclo los tres dataframes, el de Mexico, el de Japón (de las API) y de Rusia (del DataSet)
dfthree=pd.merge(dftwo,Japan_per_year_mean, on=['Year','Year'])

In [21]:
urlmx='https://datosmacro.expansion.com/demografia/poblacion/mexico'
html=requests.get(urlmx).content
soup=bs(html, 'html.parser')
#Años MX
cont=soup.find_all('td',{'class':'fecha'})
añomx=[]
for e in range(0,9):
    añomx.append(cont[e].string)
añomx.reverse()
añomx=[int(f) for f in añomx]
#Poblacion MX
cont1=soup.find_all('td',{'class':'numero'})
pobmx=[]
for e in range(3,39,4):
    pobmx.append(cont1[e]['data-value'])
pobmx.reverse()
pobmx=[int(e)/1e6 for e in pobmx]

In [22]:
#DataFrame de la poblacion en Mexico del año 2010 al 2019
Mexico_population_per_year=pd.DataFrame(list(zip(añomx,pobmx)),columns =['Year','MX population [Millions]']) 

In [23]:
#Mezclo los cuatro dataframes, el de Mexico, el de Japón (de las API) y de Rusia (del DataSet) 
#y la Poblacion de Mexico (WebScraping)
DF=pd.merge(dfthree,Mexico_population_per_year, on=['Year','Year'])
display(HTML('<h2>Consumo de electricidad promedio por año (billones de kWh) y Población Mexicana</h2>'))
display_html(DF)

,Year,Mexico,Russia,Japan,MX population [Millions]
0,2010,216.203627,856.338168,1019.98496,114.256
1,2011,240.301962,898.790326,1012.20742,115.683
2,2012,246.164140,941.242485,945.86714,117.054
3,2013,233.195690,983.694643,967.20916,118.395
4,2014,238.057060,1026.146801,950.56874,119.713
5,2015,246.842840,1068.598960,943.70264,121.006
6,2016,258.666440,1111.051118,943.68252,122.273
7,2017,270.490040,1153.503276,931.92926,123.518
8,2018,282.313640,1195.955435,924.18466,124.738
